In [1]:
from csv import reader;
import numpy as np;
import scipy.sparse.linalg as linalg;
import pandas as pd;
import json;

### 추천 문제를 받을 학생의 문제풀이 데이터 입력

In [2]:
student = input("학생 id를 입력하세요: ");
units = input("문제 소단원 리스트를 입력하세요: ");
results = input("정오답 리스트를 입력하세요: ");

test_solvings = {"student": student, "units": json.loads(units), "results": json.loads(results)}
print(test_solvings)

학생 id를 입력하세요: A050001610
문제 소단원 리스트를 입력하세요: [1353,1353,1353,1353,1353,1353,1353,314,315,315,315,314,315,315,1098,1104,1098,1098,1098,1104,1104,1101,1102,1102,1101,1102,1102,1102,1105,1105,1105,1105,1105,1105,1105,1106,1106,1106,1106,1106,1106,1106,1107,1107,1107,1107,1107,1107,1107,1468,1468,1468,1468,1468,1469,1469,1467,1467,1467,1467,1468,1469,1469,1470,1470,1468,1468,1468,1468,1470,1408,1408,1408,1408,1408,1408,1408,1410,1410,1410,1410,1410,1410,1410,1373,1373,1372,1373,1372,1373,1372,1375,1374,1374,1374,1375,1374,1374,1376,1376,1376,1376,1376,1376,1376,1379,1379,1379,1379,1379,1379,1379,1377,1377,1377,1377,1377,1377,1377,1378,1378,1378,1378,1378,1378,1378,1378,1379,1379,1377,1378,1377,1379,293,293,293,293,293,295,295,296,294,296,296,294,294,294,297,297,297,298,297,298,297 ]
정오답 리스트를 입력하세요: [1,1,1,1,1,1,0,1,0,0,1,1,0,0,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,0,1,0,1,0,0,1,1,1,1,0,0,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,

### 소단원 정보 추출

In [3]:
import json;

with open("./assets/math_unit_tree.json", "r", encoding="UTF-8") as json_file:
    math_unit_tree = json.load(json_file);

max_id = 0;

for key, value in math_unit_tree.items():
    max_id = max(value["fromConcept"]["id"], value["toConcept"]["id"], max_id);

math_units = [None] * (max_id + 1)

for key, value in math_unit_tree.items():
    for conceptKey, conceptValue in value.items():
        math_units[conceptValue["id"]] = conceptValue["name"]

df_math_units = pd.DataFrame(math_units, columns=["name"], dtype=np.str0);
df_math_units.index.name= "unitId"
df_math_units["name"] = df_math_units["name"].astype(str);

# df_math_units.to_csv('./math_units.csv', sep=',');
df_math_units = pd.read_csv('./math_units.csv', sep=',')
df_math_units.drop("unitId", inplace=True, axis=1)
df_math_units.index.name= "unitId"

print(df_math_units)

                    name
unitId                  
0                   None
1                   None
2                   None
3       지수가 자연수일 때의 지수법칙
4                  거듭제곱근
...                  ...
11267               None
11268               None
11269    천만 단위까지의 수 알아보기
11270    천억 단위까지의 수 알아보기
11271    천조 단위까지의 수 알아보기

[11272 rows x 1 columns]


### 학습 데이터 - 문제풀이 데이터 파일 읽기
### 사용자 입력 문제풀이 데이터 추가

In [4]:
solvings = [];

row_cnt = 0;

# open file in read mode
with open("./assets/solving-train.csv", 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        if row_cnt % 3 == 0:
                student = row[0];
        elif row_cnt % 3 == 1:
                units = list(map(int, row));
        elif row_cnt % 3 == 2:
                results = list(map(int, row));
                solvings.append({"student": student, "units": units, "results": results });
        row_cnt += 1;

solvings.append(test_solvings)

### 사용자의 단원별 정답률 행렬로 변환

In [5]:
max_unit_idx = 0;

for solving in solvings:
    max_unit_idx = max(solving["units"] + [max_unit_idx]);

# row: students
# column: problems
correct_rate_table = [];

for solving in solvings:
    corrects = [0] * (max_unit_idx + 1);
    num_problems = [0] * (max_unit_idx + 1);
    
    for result_idx in range(len(solving["results"])):
        unit = solving["units"][result_idx];
        corrects[unit] += solving["results"][result_idx];
        num_problems[unit] += 1; 
        
    correct_rates = list(map(lambda i: 0 if num_problems[i] == 0 else corrects[i] / num_problems[i] ,range(len(corrects))))
    correct_rate_table.append(correct_rates);

students = list(map(lambda solving: solving["student"], solvings))

df_user_unit_results = pd.DataFrame(correct_rate_table, index=students);
df_user_unit_results.index.name="userId"
df_user_unit_results.columns.name="unitId"
df_user_unit_results.head()

matrix = df_user_unit_results.values

print(matrix)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.66666667 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


### SVD 행렬분해

In [6]:
U, sigma, Vt = linalg.svds(matrix, k=12);

print(U.shape)
print(sigma.shape)
print(Vt.shape)

sigma = np.diag(sigma);
print(sigma.shape)

(14879, 12)
(12,)
(12, 1859)
(12, 12)


### 예측행렬 생성

In [7]:
svd_predicated_ratings = np.dot(np.dot(U,sigma), Vt)
df_svd_predictions = pd.DataFrame(svd_predicated_ratings, index=students)

print(df_svd_predictions)

                    0             1             2             3     \
A010002020 -6.199803e-19  1.954936e-02  1.911582e-02  2.051996e-02   
A010000731 -6.144881e-18  1.937618e-01  1.894648e-01  2.033818e-01   
A020002429  1.485844e-18 -7.315189e-18 -1.464968e-17 -9.185017e-18   
A060001077  8.291334e-19  1.794114e-17  1.487521e-17  1.536650e-17   
A070001916 -9.765709e-19 -2.885801e-17 -1.011054e-17 -5.987852e-17   
...                  ...           ...           ...           ...   
A030001718 -2.911673e-18 -4.156887e-17 -1.278821e-16 -2.242818e-17   
A050000647 -1.859227e-18 -6.436722e-17 -2.466894e-17 -6.073645e-17   
A010001379 -1.479390e-17  4.664848e-01  4.561395e-01  4.896451e-01   
A030002321 -2.363193e-18 -4.149432e-17 -1.136987e-16 -2.640164e-17   
A050001610 -3.444597e-18 -1.366461e-16 -6.271123e-17 -1.307829e-16   

                    4             5             6             7     \
A010002020  1.758672e-02  2.016702e-02 -1.302879e-18  1.939181e-02   
A010000731  1.74309

### 정렬하여 예측행렬에서 추천 단원 도출
사용자가 풀었던 단원은 추천에서 제외

In [8]:
def recommended_units(svd_prediction, user_id, origin_units_df, origin_ratings_df, num_recommendations=5):
    sorted_user_predictions = svd_prediction.loc[user_id].sort_values(ascending=False)
    sorted_user_predictions.index.name = "unitId"
    sorted_user_predictions.name = "correctRate"

    user_data = origin_ratings_df.loc[user_id]
    user_history = user_data.to_frame(name="correctRate").merge(origin_units_df, on = "unitId").sort_values(["correctRate"],ascending=False);

    recommendations = origin_units_df[~origin_units_df.isin(user_history.index)]
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), on="unitId")
    recommendations = recommendations.rename(columns={"correctRate": "Predictions"}).sort_values("Predictions", ascending=False)

    return user_history, recommendations



### 단원 추천 결과

In [9]:
already_rated, predictions = recommended_units(df_svd_predictions, test_solvings["student"], df_math_units, df_user_unit_results, 5);

print(predictions.head(5));

      unitId              name  Predictions
1375    1375               정적분     0.338199
1353    1353             치환적분법     0.317584
1372    1372     부정적분과 도함수의 관계     0.317528
1373    1373  함수 $y=x^n$의 부정적분     0.317340
1377    1377         정적분과 무한급수     0.311553
